In [1]:
# ✅ Compatible & conflict-free versions (CUDA 11.8 / CPU safe)
!pip install --no-cache-dir \
    torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0 \
    ultralytics==8.2.41 \
    segmentation-models-pytorch==0.3.3 \
    timm==0.9.2 \
    albumentations==1.3.1 \
    opencv-python==4.8.1.78 \
    numpy==1.24.4 scipy==1.11.3 scikit-image==0.21.0 \
    tqdm==4.66.1 pyyaml==6.0.1 \
    pycocotools==2.0.7


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 192.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 238.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 270.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 216.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 274.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 213.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 209.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 174.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.0/793.0 kB 344.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Imports
import os
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import Counter, defaultdict
import re
import random
from pathlib import Path
import shutil

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

from ultralytics import YOLO
import segmentation_models_pytorch as smp
from scipy.ndimage import distance_transform_edt
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [3]:
class Config:
    # Paths
    DATA_DIR = '/kaggle/input/solafune-treecanopy'
    TRAIN_IMG_DIR = os.path.join(DATA_DIR, 'train_images/train_images')
    EVAL_IMG_DIR = os.path.join(DATA_DIR, 'evaluation_images/evaluation_images')
    TRAIN_ANN_PATH = os.path.join(DATA_DIR, 'train_annotations.json')
    OUTPUT_DIR = './outputs_phase2'
    SUBMISSION_PATH = './submission_phase2.json'
    
    # Multi-Model Ensemble
    TRAIN_YOLO = True
    TRAIN_MASKRCNN = True  # Set True if you have resources
    TRAIN_UNETPP = True
    
    # YOLOv8 Settings
    YOLO_MODEL = 'yolov8x-seg.pt'
    YOLO_EPOCHS = 150
    YOLO_PATIENCE = 50
    
    # U-Net++ Settings
    UNETPP_ARCHITECTURE = 'UnetPlusPlus'
    UNETPP_ENCODER = 'efficientnet-b7'
    UNETPP_EPOCHS = 150
    UNETPP_BATCH_SIZE = 1
    UNETPP_IMG_SIZE = 512
    
    # Training
    SEED = 42
    VAL_SPLIT = 0.15
    
    # Multi-scale settings
    TRAIN_SCALES = [512, 640]
    INFER_SCALES = [512, 640]
    
    # TTA Settings
    USE_TTA = True
    TTA_SCALES = [0.75, 1.0, 1.25]
    TTA_FLIPS = ['none', 'hflip', 'vflip', 'both']
    
    # Ensemble weights (sum to 1.0)
    ENSEMBLE_WEIGHTS = {
        'yolov8': 0.5,  # Best overall
        'unetpp': 0.5,  # Strong semantic understanding
    }
    
    # Pseudo-labeling
    USE_PSEUDO_LABELING = True
    PSEUDO_CONFIDENCE_THRESHOLD = 0.90
    PSEUDO_ITERATIONS = 2
    
    # Resolution-aware thresholds
    THRESHOLDS_BY_SCENE_RES = {
        ('agriculture_plantation', 80): {'individual_tree': 0.02, 'group_of_trees': 0.03},
        ('agriculture_plantation', 60): {'individual_tree': 0.03, 'group_of_trees': 0.05},
        ('agriculture_plantation', 40): {'individual_tree': 0.05, 'group_of_trees': 0.08},
        ('urban_area', 80): {'individual_tree': 0.05, 'group_of_trees': 0.08},
        ('urban_area', 60): {'individual_tree': 0.06, 'group_of_trees': 0.09},
        ('urban_area', 40): {'individual_tree': 0.08, 'group_of_trees': 0.10},
        ('urban_area', 20): {'individual_tree': 0.08, 'group_of_trees': 0.12},
        ('urban_area', 10): {'individual_tree': 0.10, 'group_of_trees': 0.12},
    }
    
    # Fallback thresholds
    THRESHOLDS_BY_RES = {
        10: {'individual_tree': 0.10, 'group_of_trees': 0.12},
        20: {'individual_tree': 0.08, 'group_of_trees': 0.12},
        40: {'individual_tree': 0.08, 'group_of_trees': 0.10},
        60: {'individual_tree': 0.05, 'group_of_trees': 0.08},
        80: {'individual_tree': 0.03, 'group_of_trees': 0.05},
    }
    
    # Watershed settings (resolution-dependent)
    WATERSHED_MIN_DISTANCE = {
        10: 25,
        20: 20,
        40: 15,
        60: 10,
        80: 8
    }

os.makedirs(Config.OUTPUT_DIR, exist_ok=True)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed(Config.SEED)
print("✓ Phase 2 Configuration loaded")
print(f"  Models to train: YOLOv8={Config.TRAIN_YOLO}, U-Net++={Config.TRAIN_UNETPP}")
print(f"  Multi-scale inference: {Config.INFER_SCALES}")
print(f"  Pseudo-labeling: {Config.USE_PSEUDO_LABELING}")

✓ Phase 2 Configuration loaded
  Models to train: YOLOv8=True, U-Net++=True
  Multi-scale inference: [512, 640]
  Pseudo-labeling: True


In [4]:
# Advanced augmentation pipeline
advanced_train_augs = A.Compose([
    # Geometric
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=45, p=0.5),
    A.ElasticTransform(alpha=1, sigma=50, p=0.3),
    
    # Photometric
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    A.RandomGamma(gamma_limit=(80, 120), p=0.5),
    A.CLAHE(clip_limit=4.0, p=0.5),
    
    # Occlusion
    A.CoarseDropout(max_holes=8, max_height=64, max_width=64, p=0.5),
    A.GridDropout(ratio=0.2, p=0.3),
    
    # Weather/Environmental
    A.RandomFog(fog_coef_lower=0.1, fog_coef_upper=0.3, p=0.2),
    A.RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=2, p=0.3),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
    
    # Blur
    A.OneOf([
        A.MotionBlur(blur_limit=5, p=1),
        A.GaussianBlur(blur_limit=5, p=1),
    ], p=0.3),
    
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

val_augs = A.Compose([
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

# MixUp implementation
def mixup(img1, mask1, img2, mask2, alpha=0.4):
    """MixUp augmentation"""
    lam = np.random.beta(alpha, alpha)
    mixed_img = lam * img1 + (1 - lam) * img2
    mixed_mask = lam * mask1 + (1 - lam) * mask2
    return mixed_img, mixed_mask

# CutMix implementation
def cutmix(img1, mask1, img2, mask2):
    """CutMix augmentation"""
    lam = np.random.beta(1.0, 1.0)
    h, w = img1.shape[:2]
    cut_ratio = np.sqrt(1 - lam)
    cut_h, cut_w = int(h * cut_ratio), int(w * cut_ratio)
    
    cx, cy = np.random.randint(w), np.random.randint(h)
    x1 = np.clip(cx - cut_w // 2, 0, w)
    x2 = np.clip(cx + cut_w // 2, 0, w)
    y1 = np.clip(cy - cut_h // 2, 0, h)
    y2 = np.clip(cy + cut_h // 2, 0, h)
    
    img1[y1:y2, x1:x2] = img2[y1:y2, x1:x2]
    mask1[y1:y2, x1:x2] = mask2[y1:y2, x1:x2]
    return img1, mask1

print("✓ Advanced augmentations defined")
print("  Including: MixUp, CutMix, GridDropout, Weather effects")

✓ Advanced augmentations defined
  Including: MixUp, CutMix, GridDropout, Weather effects


/usr/local/lib/python3.11/dist-packages/albumentations/augmentations/blur/transforms.py:184: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  warnings.warn(


In [5]:
class FocalLoss(nn.Module):
    """Focal Loss for handling class imbalance"""
    def __init__(self, alpha=0.25, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
    
    def forward(self, pred, target):
        bce = F.binary_cross_entropy_with_logits(pred, target, reduction='none')
        pt = torch.exp(-bce)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * bce
        return focal_loss.mean()

class BoundaryLoss(nn.Module):
    """Boundary Loss for improving edge precision"""
    def forward(self, pred, target):
        # Compute distance transform
        target_np = target.cpu().numpy()
        dist_maps = []
        for t in target_np:
            if t.ndim == 3:
                t = t[0]  # Remove channel dimension if present
            dist = distance_transform_edt(1 - t)
            dist_maps.append(dist)
        dist_maps = np.stack(dist_maps)
        dist_maps = torch.from_numpy(dist_maps).unsqueeze(1).to(pred.device).float()
        
        # Boundary loss
        pred_softmax = torch.sigmoid(pred)
        boundary_loss = (dist_maps * pred_softmax).mean()
        return boundary_loss

class DiceLoss(nn.Module):
    """Dice Loss"""
    def forward(self, pred, target, smooth=1.0):
        pred = torch.sigmoid(pred)
        intersection = (pred * target).sum(dim=(2, 3))
        union = pred.sum(dim=(2, 3)) + target.sum(dim=(2, 3))
        dice = (2.0 * intersection + smooth) / (union + smooth)
        return 1.0 - dice.mean()

class CombinedLoss(nn.Module):
    """Combined Focal + Dice + Boundary Loss"""
    def __init__(self):
        super().__init__()
        self.focal = FocalLoss(alpha=0.25, gamma=2.0)
        self.dice = DiceLoss()
        self.boundary = BoundaryLoss()
    
    def forward(self, pred, target):
        return 0.4 * self.focal(pred, target) + \
               0.3 * self.dice(pred, target) + \
               0.3 * self.boundary(pred, target)

print("✓ Advanced loss functions defined")
print("  Focal Loss: Handles class imbalance")
print("  Boundary Loss: Improves edge precision")
print("  Combined: 0.4*Focal + 0.3*Dice + 0.3*Boundary")


✓ Advanced loss functions defined
  Focal Loss: Handles class imbalance
  Boundary Loss: Improves edge precision
  Combined: 0.4*Focal + 0.3*Dice + 0.3*Boundary


In [6]:
def separate_instances_watershed(mask, resolution=10):
    """
    Watershed-based instance separation for overlapping trees
    Adjusts min_distance based on image resolution
    """
    min_distance = Config.WATERSHED_MIN_DISTANCE.get(resolution, 15)
    
    # Distance transform
    distance = ndimage.distance_transform_edt(mask)
    
    # Find peaks (tree centers)
    local_max = peak_local_max(
        distance,
        min_distance=min_distance,
        labels=mask.astype(int),
        footprint=np.ones((3, 3))
    )
    
    # Create markers
    markers = np.zeros_like(mask, dtype=int)
    markers[tuple(local_max.T)] = np.arange(len(local_max)) + 1
    markers = ndimage.label(markers)[0]
    
    # Apply watershed
    labels = watershed(-distance, markers, mask=mask)
    
    return labels

print("✓ Watershed instance separation defined")
print(f"  Min distances by resolution: {Config.WATERSHED_MIN_DISTANCE}")

✓ Watershed instance separation defined
  Min distances by resolution: {10: 25, 20: 20, 40: 15, 60: 10, 80: 8}


In [7]:
if Config.TRAIN_UNETPP:
    print("Training U-Net++ model...")
    
    # Clear CUDA cache before training
    import gc
    torch.cuda.empty_cache()
    gc.collect()
    
    # Load and prepare data
    with open(Config.TRAIN_ANN_PATH, 'r') as f:
        train_data = json.load(f)
    
    # Real dataset class with proper mask generation
    class TreeCanopyDataset(Dataset):
        def __init__(self, items, img_dir, transform=None, img_size=512):
            self.items = items
            self.img_dir = img_dir
            self.transform = transform
            self.img_size = img_size
        
        def __len__(self):
            return len(self.items)
        
        def __getitem__(self, idx):
            item = self.items[idx]
            
            # Load image
            img_path = os.path.join(self.img_dir, item['file_name'])
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            h, w = image.shape[:2]
            
            # Create mask from annotations
            mask = np.zeros((2, h, w), dtype=np.float32)  # 2 classes
            
            for ann in item.get('annotations', []):
                class_name = ann['class']
                class_idx = 0 if class_name == 'individual_tree' else 1
                
                # Get polygon points
                seg = ann['segmentation']
                points = np.array(seg).reshape(-1, 2).astype(np.int32)
                
                # Fill polygon on mask
                cv2.fillPoly(mask[class_idx], [points], 1.0)
            
            # Resize
            image = cv2.resize(image, (self.img_size, self.img_size))
            mask_resized = np.zeros((2, self.img_size, self.img_size), dtype=np.float32)
            mask_resized[0] = cv2.resize(mask[0], (self.img_size, self.img_size))
            mask_resized[1] = cv2.resize(mask[1], (self.img_size, self.img_size))
            
            # Apply transforms
            if self.transform is not None:
                # Albumentations expects HWC for image and HW for mask
                # We need to handle 2-channel mask specially
                transformed = self.transform(image=image, mask=mask_resized.transpose(1, 2, 0))
                image = transformed['image']
                mask_resized = transformed['mask'].permute(2, 0, 1)  # Back to CHW
            else:
                image = torch.from_numpy(image.transpose(2, 0, 1)).float() / 255.0
                mask_resized = torch.from_numpy(mask_resized).float()
            
            return image, mask_resized
    
    # Split data into train/val (80/20)
    random.shuffle(train_data['images'])
    split_idx = int(len(train_data['images']) * 0.8)
    train_items = train_data['images'][:split_idx]
    val_items = train_data['images'][split_idx:]
    
    print(f"✓ Data split: {len(train_items)} train, {len(val_items)} val")
    
    # Create datasets with memory-optimized size
    IMG_SIZE = 512  # Memory optimized
    train_dataset = TreeCanopyDataset(train_items, Config.TRAIN_IMG_DIR, transform=advanced_train_augs, img_size=IMG_SIZE)
    val_dataset = TreeCanopyDataset(val_items, Config.TRAIN_IMG_DIR, transform=val_augs, img_size=IMG_SIZE)
    
    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=1,  # Memory optimized
        shuffle=True,
        num_workers=0,
        pin_memory=False
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=0,
        pin_memory=False
    )
    
    # Initialize model with memory-optimized encoder
    print("⚠️  Memory optimization: Using efficientnet-b3 instead of b7")
    print("⚠️  To use b7, ensure you have 24GB+ GPU or reduce batch size/image size further")
    
    unetpp_model = smp.UnetPlusPlus(
        encoder_name='efficientnet-b3',  # Memory optimized
        encoder_weights='imagenet',
        in_channels=3,
        classes=2,
        activation=None
    )
    
    # Enable gradient checkpointing for memory efficiency
    if hasattr(unetpp_model.encoder, 'set_grad_checkpointing'):
        unetpp_model.encoder.set_grad_checkpointing(True)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    unetpp_model = unetpp_model.to(device)
    
    print(f"✓ U-Net++ model initialized")
    print(f"  Encoder: efficientnet-b3 (memory optimized)")
    print(f"  Parameters: {sum(p.numel() for p in unetpp_model.parameters()):,}")
    print(f"  Train batches: {len(train_loader)}, Val batches: {len(val_loader)}")
    print(f"  Image size: {IMG_SIZE}x{IMG_SIZE}")
    print(f"  Batch size: 1 (memory optimized)")
    
    # Loss and optimizer
    criterion = CombinedLoss()
    optimizer = torch.optim.AdamW(unetpp_model.parameters(), lr=3e-4, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=20, T_mult=2)
    
    # AMP for memory efficiency
    scaler = torch.cuda.amp.GradScaler()
    print("✓ Automatic Mixed Precision (AMP) enabled for memory efficiency")
    
    # IMPROVED: Use Dice Score instead of IoU (more stable)
    def validate_model(model, val_loader, device):
        """
        Calculate validation Dice score (more stable than IoU)
        Dice = 2 * |X ∩ Y| / (|X| + |Y|)
        """
        model.eval()
        
        total_dice = 0.0
        count = 0
        
        with torch.no_grad():
            for images, masks in val_loader:
                images = images.to(device)
                masks = masks.to(device)
                
                with torch.cuda.amp.autocast():
                    outputs = model(images)
                    preds = torch.sigmoid(outputs)
                
                # Calculate Dice for each sample and class
                for b in range(images.size(0)):
                    for c in range(2):
                        # Use adaptive threshold based on prediction distribution
                        pred_flat = preds[b, c].flatten()
                        threshold = pred_flat.median().item()  # Adaptive threshold
                        threshold = max(0.3, min(0.7, threshold))  # Clamp between 0.3-0.7
                        
                        pred_mask = (preds[b, c] > threshold).float()
                        true_mask = masks[b, c]
                        
                        # Skip if both are completely empty
                        if true_mask.sum() == 0 and pred_mask.sum() == 0:
                            continue
                        
                        # Calculate Dice coefficient
                        intersection = (pred_mask * true_mask).sum()
                        dice_denominator = pred_mask.sum() + true_mask.sum()
                        
                        if dice_denominator > 0:
                            dice = (2.0 * intersection / dice_denominator).item()
                            total_dice += dice
                            count += 1
                        else:
                            # Both predictions and ground truth are empty
                            # This is actually a correct prediction (empty -> empty)
                            total_dice += 1.0
                            count += 1
        
        mean_dice = total_dice / count if count > 0 else 0.0
        return mean_dice
    
    # Training loop with gradient clipping
    best_dice = 0.0
    patience = 30
    patience_counter = 0
    epochs = 150  # Demo - use 150 in production
    
    print(f"⚠️  Running {epochs} epochs for demo (set to 150 in production)")
    print(f"📊 Using Dice Score as metric (more stable than IoU)")
    
    for epoch in range(epochs):
        unetpp_model.train()
        epoch_loss = 0.0
        
        for batch_idx, (images, masks) in enumerate(train_loader):
            images = images.to(device)
            masks = masks.to(device)
            
            optimizer.zero_grad()
            
            # Mixed precision forward pass
            with torch.cuda.amp.autocast():
                outputs = unetpp_model(images)
                loss = criterion(outputs, masks)
            
            # Backward with gradient scaling
            scaler.scale(loss).backward()
            
            # Gradient clipping to prevent exploding gradients
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(unetpp_model.parameters(), max_norm=1.0)
            
            scaler.step(optimizer)
            scaler.update()
            
            epoch_loss += loss.item()
            
            # Clear cache periodically
            if batch_idx % 5 == 0:
                torch.cuda.empty_cache()
        
        avg_loss = epoch_loss / len(train_loader)
        
        # Validation
        val_dice = validate_model(unetpp_model, val_loader, device)
        
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}, Val Dice: {val_dice:.4f}")
        
        # Save best model
        if val_dice > best_dice:
            best_dice = val_dice
            torch.save(unetpp_model.state_dict(), os.path.join(Config.OUTPUT_DIR, 'best_unetpp.pt'))
            patience_counter = 0
            print(f"  ✓ New best model saved! Dice: {best_dice:.4f}")
        else:
            patience_counter += 1
        
        # Early stopping
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
        
        scheduler.step()
    
    print(f"✓ U-Net++ training complete. Best Dice: {best_dice:.4f}")
    
else:
    print("⚠️  U-Net++ training skipped (set Config.TRAIN_UNETPP=True to enable)")

# Memory tip
print("\n💡 Memory Management Tips:")
print("1. If still OOM, set Config.TRAIN_UNETPP = False")
print("2. Or further reduce image size to 384x384")
print("3. Or use encoder='efficientnet-b0' (smallest)")
print("4. For production with 24GB+ GPU, use efficientnet-b7")
if torch.cuda.is_available():
    print(f"\nCurrent GPU memory: {torch.cuda.memory_allocated()/1024**3:.2f} GB / {torch.cuda.max_memory_allocated()/1024**3:.2f} GB peak")

Training U-Net++ model...
✓ Data split: 120 train, 30 val
⚠️  Memory optimization: Using efficientnet-b3 instead of b7
⚠️  To use b7, ensure you have 24GB+ GPU or reduce batch size/image size further


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth
100%|██████████| 47.1M/47.1M [00:00<00:00, 125MB/s]


✓ U-Net++ model initialized
  Encoder: efficientnet-b3 (memory optimized)
  Parameters: 13,624,938
  Train batches: 120, Val batches: 30
  Image size: 512x512
  Batch size: 1 (memory optimized)
✓ Automatic Mixed Precision (AMP) enabled for memory efficiency
⚠️  Running 150 epochs for demo (set to 150 in production)
📊 Using Dice Score as metric (more stable than IoU)
Epoch 1/150 - Loss: 2.3276, Val Dice: 0.0978
  ✓ New best model saved! Dice: 0.0978
Epoch 2/150 - Loss: 0.6155, Val Dice: 0.0000
Epoch 3/150 - Loss: 0.4125, Val Dice: 0.0000
Epoch 4/150 - Loss: 0.3916, Val Dice: 0.0003
Epoch 5/150 - Loss: 0.3841, Val Dice: 0.0020
Epoch 6/150 - Loss: 0.3823, Val Dice: 0.0003
Epoch 7/150 - Loss: 0.3811, Val Dice: 0.0043
Epoch 8/150 - Loss: 0.3794, Val Dice: 0.0083
Epoch 9/150 - Loss: 0.3738, Val Dice: 0.0223
Epoch 10/150 - Loss: 0.3766, Val Dice: 0.0068
Epoch 11/150 - Loss: 0.3656, Val Dice: 0.0475
Epoch 12/150 - Loss: 0.3615, Val Dice: 0.0831
Epoch 13/150 - Loss: 0.3587, Val Dice: 0.0686
Epo

In [8]:
from torchvision.transforms.functional import normalize

In [9]:
class EnsembleModel:
    """Ensemble multiple models with weighted averaging"""
    
    def __init__(self, model_paths, weights, unetpp_encoder='efficientnet-b3'):
        self.models = []
        self.weights = weights
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # Load YOLOv8
        if 'yolov8' in model_paths and os.path.exists(model_paths['yolov8']):
            self.yolo_model = YOLO(model_paths['yolov8'])
            print(f"✓ Loaded YOLOv8 from {model_paths['yolov8']}")
        else:
            self.yolo_model = None
        
        # Load U-Net++
        if 'unetpp' in model_paths and os.path.exists(model_paths['unetpp']):
            # Use the encoder that was actually used during training
            print(f"⚠️  Loading U-Net++ with encoder: {unetpp_encoder}")
            self.unetpp_model = smp.UnetPlusPlus(
                encoder_name=unetpp_encoder,  # Use parameter instead of Config
                encoder_weights=None,
                in_channels=3,
                classes=2
            )
            self.unetpp_model.load_state_dict(torch.load(model_paths['unetpp']))
            self.unetpp_model.to(self.device)
            self.unetpp_model.eval()
            print(f"✓ Loaded U-Net++ from {model_paths['unetpp']}")
        else:
            self.unetpp_model = None
    
    def predict_ensemble(self, image, use_tta=True):
        """
        Ensemble prediction with weighted averaging
        Returns combined probability maps
        """
        h, w = image.shape[:2]
        ensemble_pred = np.zeros((2, h, w), dtype=np.float32)
        total_weight = 0.0
        
        # YOLOv8 prediction
        if self.yolo_model is not None:
            yolo_pred = self._predict_yolo(image, use_tta)
            ensemble_pred += self.weights['yolov8'] * yolo_pred
            total_weight += self.weights['yolov8']
        
        # U-Net++ prediction
        if self.unetpp_model is not None:
            unetpp_pred = self._predict_unetpp(image, use_tta)
            ensemble_pred += self.weights['unetpp'] * unetpp_pred
            total_weight += self.weights['unetpp']
        
        # Normalize by total weight
        if total_weight > 0:
            ensemble_pred /= total_weight
        
        return ensemble_pred
    
    def _predict_yolo(self, image, use_tta):
        """YOLOv8 prediction with proper mask extraction"""
        h, w = image.shape[:2]
        prob_maps = np.zeros((2, h, w), dtype=np.float32)
        
        # Run YOLOv8 inference
        results = self.yolo_model.predict(image, verbose=False, imgsz=640)
        
        if len(results) > 0 and results[0].masks is not None:
            masks = results[0].masks.data.cpu().numpy()
            classes = results[0].boxes.cls.cpu().numpy()
            
            for mask, cls in zip(masks, classes):
                # Resize mask to original size
                mask_resized = cv2.resize(mask, (w, h))
                
                # Map class (0=individual_tree, 1=group_of_trees)
                class_idx = int(cls)
                if class_idx < 2:  # Only process valid classes
                    prob_maps[class_idx] = np.maximum(prob_maps[class_idx], mask_resized)
        
        return prob_maps
    
    def _predict_unetpp(self, image, use_tta):
        """U-Net++ prediction with TTA support"""
        h, w = image.shape[:2]
        
        if use_tta:
            # TTA for U-Net++
            all_preds = []
            for scale in Config.TTA_SCALES:
                scaled_h, scaled_w = int(h * scale), int(w * scale)
                scaled_img = cv2.resize(image, (scaled_w, scaled_h))
                
                # Normalize
                img_tensor = torch.from_numpy(scaled_img.transpose(2, 0, 1)).float() / 255.0

                img_tensor = normalize(img_tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                img_tensor = img_tensor.unsqueeze(0).to(self.device)
                
                # Predict
                with torch.no_grad():
                    pred = self.unetpp_model(img_tensor)
                    pred = torch.sigmoid(pred).cpu().numpy()[0]
                
                # Resize back to original size
                pred_resized = np.zeros((2, h, w), dtype=np.float32)
                pred_resized[0] = cv2.resize(pred[0], (w, h))
                pred_resized[1] = cv2.resize(pred[1], (w, h))
                
                all_preds.append(pred_resized)
            
            return np.mean(all_preds, axis=0)
        else:
            # Standard prediction
            # Normalize
            img_tensor = torch.from_numpy(image.transpose(2, 0, 1)).float() / 255.0
            img_tensor = F.normalize(img_tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            img_tensor = img_tensor.unsqueeze(0).to(self.device)
            
            # Predict
            with torch.no_grad():
                pred = self.unetpp_model(img_tensor)
                pred = torch.sigmoid(pred).cpu().numpy()[0]
            
            return pred

print("✓ Ensemble model class defined with working inference methods")

✓ Ensemble model class defined with working inference methods


In [10]:
# Initialize ensemble
model_paths = {
    'yolov8': os.path.join(Config.OUTPUT_DIR, 'yolov8x_training/weights/best.pt'),
    'unetpp': os.path.join(Config.OUTPUT_DIR, 'best_unetpp.pt')
}

# 🔧 FIX: Specify the encoder used during training (efficientnet-b3 for memory optimization)
# Change to 'efficientnet-b7' if you trained with that encoder
ensemble = EnsembleModel(model_paths, Config.ENSEMBLE_WEIGHTS, unetpp_encoder='efficientnet-b3')

# Helper function
def infer_scene_type(filename):
    fn_lower = filename.lower()
    if 'agri' in fn_lower or 'plantation' in fn_lower:
        return 'agriculture_plantation'
    elif 'urban' in fn_lower or 'city' in fn_lower:
        return 'urban_area'
    elif 'rural' in fn_lower:
        return 'rural_area'
    elif 'industrial' in fn_lower:
        return 'industrial_area'
    elif 'field' in fn_lower:
        return 'open_field'
    else:
        m = re.match(r'^(\d+)cm_', filename)
        if m:
            res = int(m.group(1))
            return 'agriculture_plantation' if res >= 40 else 'urban_area'
        return 'rural_area'

# Get evaluation images
eval_files = sorted([f for f in os.listdir(Config.EVAL_IMG_DIR) if f.lower().endswith('.tif')])
print(f"Found {len(eval_files)} evaluation images")

# Generate submission
submission = {'images': []}

for filename in tqdm(eval_files, desc="Ensemble Inference with TTA"):
    img_path = os.path.join(Config.EVAL_IMG_DIR, filename)
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w = image.shape[:2]
    
    # Extract metadata
    m = re.match(r'^(\d+)cm_', filename)
    cm_resolution = int(m.group(1)) if m else 10
    scene_type = infer_scene_type(filename)
    
    # Get scene+resolution specific threshold
    threshold_key = (scene_type, cm_resolution)
    if threshold_key in Config.THRESHOLDS_BY_SCENE_RES:
        thresholds = Config.THRESHOLDS_BY_SCENE_RES[threshold_key]
    else:
        thresholds = Config.THRESHOLDS_BY_RES.get(cm_resolution, Config.THRESHOLDS_BY_RES[10])
    
    # Ensemble prediction with TTA
    probs = ensemble.predict_ensemble(image, use_tta=Config.USE_TTA)
    
    annotations = []
    
    # Process each class
    for class_idx, class_name in enumerate(['individual_tree', 'group_of_trees']):
        threshold = thresholds[class_name]
        
        # Threshold
        mask = (probs[class_idx] > threshold).astype(np.uint8)
        
        # Watershed instance separation
        labels = separate_instances_watershed(mask, cm_resolution)
        
        # Extract instances
        for instance_id in range(1, labels.max() + 1):
            instance_mask = (labels == instance_id).astype(np.uint8)
            
            # Find contours
            contours, _ = cv2.findContours(instance_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            for contour in contours:
                if len(contour) < 3:
                    continue
                
                # Simplify
                epsilon = 0.2
                contour_simplified = cv2.approxPolyDP(contour, epsilon, True)
                
                if len(contour_simplified) < 3:
                    continue
                
                # Convert to segmentation format
                polygon = contour_simplified.reshape(-1, 2).astype(int).tolist()
                segmentation = [coord for pt in polygon for coord in pt]
                
                # Calculate confidence
                confidence = float(probs[class_idx][instance_mask > 0].mean()) if instance_mask.sum() > 0 else 0.5
                
                annotations.append({
                    'class': class_name,
                    'confidence_score': confidence,
                    'segmentation': segmentation
                })
    
    submission['images'].append({
        'file_name': filename,
        'width': w,
        'height': h,
        'cm_resolution': cm_resolution,
        'scene_type': scene_type,
        'annotations': annotations
    })

# Save submission
with open(Config.SUBMISSION_PATH, 'w') as f:
    json.dump(submission, f, indent=4)

print(f"\n✓ Phase 2 submission saved to {Config.SUBMISSION_PATH}")
print(f"  Total images: {len(submission['images'])}")
print(f"  Total annotations: {sum(len(img['annotations']) for img in submission['images'])}")
print(f"  Ensemble models used: {list(Config.ENSEMBLE_WEIGHTS.keys())}")
print(f"  TTA enabled: {Config.USE_TTA}")

⚠️  Loading U-Net++ with encoder: efficientnet-b3
✓ Loaded U-Net++ from ./outputs_phase2/best_unetpp.pt
Found 150 evaluation images


Ensemble Inference with TTA: 100%|██████████| 150/150 [03:47<00:00,  1.52s/it]



✓ Phase 2 submission saved to ./submission_phase2.json
  Total images: 150
  Total annotations: 64358
  Ensemble models used: ['yolov8', 'unetpp']
  TTA enabled: True


In [11]:
def visualize_random_samples(num_samples=20, dataset_name="tree_train"):
    """Visualize random images with annotations from registered datasets"""
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches
    from matplotlib.patches import Polygon as MPLPolygon
    import random
    import cv2
    
    # Get dataset
    dataset_dicts = DatasetCatalog.get(dataset_name)
    metadata = MetadataCatalog.get(dataset_name)
    
    # Randomly sample images
    sampled = random.sample(dataset_dicts, min(num_samples, len(dataset_dicts)))
    
    # Calculate grid size
    cols = 5
    rows = (num_samples + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(20, 4 * rows))
    axes = axes.flatten() if num_samples > 1 else [axes]
    
    class_colors = {
        0: (0, 1, 0, 0.5),      # individual_tree - green
        1: (0, 0.5, 1, 0.5)     # group_of_trees - blue
    }
    
    for idx, (record, ax) in enumerate(zip(sampled, axes)):
        # Read image
        img = cv2.imread(record["file_name"])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Display image
        ax.imshow(img)
        ax.set_title(f"{os.path.basename(record['file_name'])}\n{len(record['annotations'])} trees", 
                     fontsize=8)
        ax.axis('off')
        
        # Draw annotations
        for anno in record["annotations"]:
            # Draw bounding box
            bbox = anno["bbox"]
            rect = patches.Rectangle(
                (bbox[0], bbox[1]), 
                bbox[2] - bbox[0], 
                bbox[3] - bbox[1],
                linewidth=1.5, 
                edgecolor=class_colors[anno["category_id"]][:3], 
                facecolor='none',
                alpha=0.8
            )
            ax.add_patch(rect)
            
            # Draw segmentation polygon
            if "segmentation" in anno and len(anno["segmentation"]) > 0:
                seg = anno["segmentation"][0]
                poly_coords = [(seg[i], seg[i+1]) for i in range(0, len(seg), 2)]
                polygon = MPLPolygon(
                    poly_coords, 
                    closed=True,
                    edgecolor=class_colors[anno["category_id"]][:3],
                    facecolor=class_colors[anno["category_id"]],
                    linewidth=1,
                    alpha=0.4
                )
                ax.add_patch(polygon)
            
            # Add class label
            class_name = metadata.thing_classes[anno["category_id"]]
            ax.text(
                bbox[0], bbox[1] - 5,
                class_name[:4],  # Abbreviated
                fontsize=6,
                color='white',
                bbox=dict(facecolor=class_colors[anno["category_id"]][:3], 
                         alpha=0.7, pad=1, edgecolor='none')
            )
    
    # Hide extra subplots
    for idx in range(len(sampled), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.suptitle(f"🌲 Random Sample: {len(sampled)} Images from {dataset_name}", 
                 fontsize=14, y=1.002, fontweight='bold')
    plt.show()
    
    # Print statistics
    total_annotations = sum(len(record["annotations"]) for record in sampled)
    individual_count = sum(
        sum(1 for anno in record["annotations"] if anno["category_id"] == 0)
        for record in sampled
    )
    group_count = sum(
        sum(1 for anno in record["annotations"] if anno["category_id"] == 1)
        for record in sampled
    )
    
    print(f"\n📊 Sample Statistics:")
    print(f"  Total images: {len(sampled)}")
    print(f"  Total annotations: {total_annotations}")
    print(f"  Individual trees: {individual_count} ({individual_count/total_annotations*100:.1f}%)")
    print(f"  Group of trees: {group_count} ({group_count/total_annotations*100:.1f}%)")
    print(f"  Avg annotations per image: {total_annotations/len(sampled):.1f}")


def visualize_yolo_predictions(num_samples=20, model_path=None):
    """Visualize YOLOv8 model predictions on random images"""
    import matplotlib.pyplot as plt
    import random
    import cv2
    from ultralytics import YOLO
    
    # Load model
    if model_path is None:
        model_path = os.path.join(Config.OUTPUT_DIR, 'yolov8x_training', 'weights', 'best.pt')
    
    if not os.path.exists(model_path):
        print(f"❌ Model not found: {model_path}")
        return
    
    model = YOLO(model_path)
    
    # Get random images
    img_dir = os.path.join(Config.DATA_DIR, 'train_images')
    all_images = [f for f in os.listdir(img_dir) if f.endswith('.tif')]
    sampled_images = random.sample(all_images, min(num_samples, len(all_images)))
    
    # Calculate grid size
    cols = 5
    rows = (num_samples + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(20, 4 * rows))
    axes = axes.flatten() if num_samples > 1 else [axes]
    
    total_detections = 0
    
    for idx, (img_name, ax) in enumerate(zip(sampled_images, axes)):
        img_path = os.path.join(img_dir, img_name)
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Run prediction
        results = model.predict(img_path, verbose=False)
        
        # Count detections
        num_detections = len(results[0].boxes) if results[0].boxes is not None else 0
        total_detections += num_detections
        
        # Plot results
        if num_detections > 0:
            # Plot with annotations
            result_img = results[0].plot()
            result_img = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
            ax.imshow(result_img)
        else:
            ax.imshow(img_rgb)
        
        ax.set_title(f"{img_name}\n{num_detections} detections", fontsize=8)
        ax.axis('off')
    
    # Hide extra subplots
    for idx in range(len(sampled_images), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.suptitle(f"🎯 YOLOv8 Predictions: {len(sampled_images)} Random Images", 
                 fontsize=14, y=1.002, fontweight='bold')
    plt.show()
    
    print(f"\n📊 YOLOv8 Statistics:")
    print(f"  Total images: {len(sampled_images)}")
    print(f"  Total detections: {total_detections}")
    print(f"  Avg detections per image: {total_detections/len(sampled_images):.1f}")


def visualize_unetpp_predictions(num_samples=20, model_path=None):
    """Visualize U-Net++ model predictions on random images"""
    import matplotlib.pyplot as plt
    import random
    import cv2
    import torch
    import segmentation_models_pytorch as smp
    
    # Load model
    if model_path is None:
        model_path = os.path.join(Config.OUTPUT_DIR, 'best_unetpp.pt')
    
    if not os.path.exists(model_path):
        print(f"❌ Model not found: {model_path}")
        return
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = smp.UnetPlusPlus(
        encoder_name='efficientnet-b3',
        encoder_weights=None,
        in_channels=3,
        classes=2
    )
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    
    # Get random images
    img_dir = os.path.join(Config.DATA_DIR, 'train_images')
    all_images = [f for f in os.listdir(img_dir) if f.endswith('.tif')]
    sampled_images = random.sample(all_images, min(num_samples, len(all_images)))
    
    # Calculate grid size
    cols = 5
    rows = (num_samples + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(20, 4 * rows))
    axes = axes.flatten() if num_samples > 1 else [axes]
    
    class_colors = [
        [0, 255, 0],      # individual_tree - green
        [0, 128, 255]     # group_of_trees - blue
    ]
    
    with torch.no_grad():
        for idx, (img_name, ax) in enumerate(zip(sampled_images, axes)):
            img_path = os.path.join(img_dir, img_name)
            img = cv2.imread(img_path)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Preprocess
            img_resized = cv2.resize(img, (Config.IMG_SIZE, Config.IMG_SIZE))
            img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).float() / 255.0
            img_tensor = img_tensor.unsqueeze(0).to(device)
            
            # Predict
            pred = model(img_tensor)
            pred = torch.sigmoid(pred).cpu().numpy()[0]
            
            # Create overlay
            overlay = img_rgb.copy()
            for class_idx in range(2):
                mask = pred[class_idx] > 0.5
                mask_resized = cv2.resize(mask.astype(np.uint8), (img_rgb.shape[1], img_rgb.shape[0]))
                overlay[mask_resized > 0] = (overlay[mask_resized > 0] * 0.6 + 
                                             np.array(class_colors[class_idx]) * 0.4).astype(np.uint8)
            
            ax.imshow(overlay)
            ax.set_title(f"{img_name}\nU-Net++", fontsize=8)
            ax.axis('off')
    
    # Hide extra subplots
    for idx in range(len(sampled_images), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.suptitle(f"🔷 U-Net++ Predictions: {len(sampled_images)} Random Images", 
                 fontsize=14, y=1.002, fontweight='bold')
    plt.show()
    
    print(f"\n📊 U-Net++ Statistics:")
    print(f"  Total images: {len(sampled_images)}")
    print(f"  Model: {model_path}")


def visualize_ensemble_predictions(num_samples=20):
    """Visualize ensemble predictions on random images"""
    import matplotlib.pyplot as plt
    import random
    import cv2
    
    # Check if models exist
    yolo_path = os.path.join(Config.OUTPUT_DIR, 'yolov8x_training', 'weights', 'best.pt')
    unetpp_path = os.path.join(Config.OUTPUT_DIR, 'best_unetpp.pt')
    
    if not os.path.exists(yolo_path):
        print(f"❌ YOLOv8 model not found: {yolo_path}")
        return
    if not os.path.exists(unetpp_path):
        print(f"❌ U-Net++ model not found: {unetpp_path}")
        return
    
    # Load ensemble
    ensemble = EnsembleModel(yolo_path, unetpp_path, unetpp_encoder='efficientnet-b3')
    
    # Get random images
    img_dir = os.path.join(Config.DATA_DIR, 'train_images')
    all_images = [f for f in os.listdir(img_dir) if f.endswith('.tif')]
    sampled_images = random.sample(all_images, min(num_samples, len(all_images)))
    
    # Calculate grid size
    cols = 5
    rows = (num_samples + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(20, 4 * rows))
    axes = axes.flatten() if num_samples > 1 else [axes]
    
    class_colors = [
        [0, 255, 0],      # individual_tree - green
        [0, 128, 255]     # group_of_trees - blue
    ]
    
    total_instances = 0
    
    for idx, (img_name, ax) in enumerate(zip(sampled_images, axes)):
        img_path = os.path.join(img_dir, img_name)
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Get ensemble prediction
        pred = ensemble.predict_ensemble(img_path, use_tta=True)
        
        # Apply watershed for instance separation
        instances = watershed_instance_separation(pred, img_rgb)
        total_instances += len(instances)
        
        # Create overlay
        overlay = img_rgb.copy()
        for inst in instances:
            mask = inst['mask']
            class_idx = inst['class']
            
            # Draw filled mask
            overlay[mask > 0] = (overlay[mask > 0] * 0.6 + 
                                np.array(class_colors[class_idx]) * 0.4).astype(np.uint8)
            
            # Draw contour
            contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(overlay, contours, -1, class_colors[class_idx], 2)
        
        ax.imshow(overlay)
        ax.set_title(f"{img_name}\n{len(instances)} trees", fontsize=8)
        ax.axis('off')
    
    # Hide extra subplots
    for idx in range(len(sampled_images), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.suptitle(f"🎯 Ensemble Predictions: {len(sampled_images)} Random Images", 
                 fontsize=14, y=1.002, fontweight='bold')
    plt.show()
    
    print(f"\n📊 Ensemble Statistics:")
    print(f"  Total images: {len(sampled_images)}")
    print(f"  Total instances: {total_instances}")
    print(f"  Avg instances per image: {total_instances/len(sampled_images):.1f}")


# Example usage (uncomment to use):
# After dataset registration:
# visualize_random_samples(num_samples=20, dataset_name="tree_train")

# After YOLOv8 training:
# visualize_yolo_predictions(num_samples=20)

# After U-Net++ training:
# visualize_unetpp_predictions(num_samples=20)

# After both models trained:
# visualize_ensemble_predictions(num_samples=20)